# MGMTSA408 -- Lecture 2: Multi-Leg Revenue Management

In this notebook, we will explore how to formulate the multi-leg capacity control problem using linear programming.

## Formulating the static multi-leg capacity control problem

Let us see how to formulate the multi-leg capacity control problem. We assume we have four legs (BOS->SLC, JFK->SLC, SLC->SFO, SLC->LAX) and eight itineraries (four that are direct flights, and four that fly from one of BOS/JFK to one of SFO/LAX with a stop in SLC). 

In [1]:
import numpy as np

nItineraries = 8
nLegs = 4

# Legs:
# 0: BOS->SLC
# 1: JFK->SLC
# 2: SLC->SFO
# 3: SLC->LAX

# Let's assume we have the following seats on the legs:
B = np.array([160, 160, 160, 160])

# Assume we are selling over 750 periods. (Again, can think
# of these as days or smaller periods than days, e.g., 12hr/6hr/2hr periods.)
T = 750

# Below is a list of lists. Each element is a list
# that specifies which legs are used in the itinerary. We'll
# use this when we define our constraints for the LP momentarily.
itineraries_to_legs = [ [0],
                        [1],
                        [2],
                        [3],
                        [0,2],
                        [0,3],
                        [1,2],
                        [1,3]]

revenue = np.array([260, 280, 250, 260, 330, 330, 340, 350])
probability = np.array([0.08, 0.09, 0.08, 0.10, 0.13, 0.14, 0.12, 0.12])

forecast = T * probability

# Formulate the LP:
from gurobipy import * 

# Create the model and the decision variables.
m = Model()

x = m.addVars(nItineraries, lb = 0, ub = forecast)

# Define the constraints.
# Notice how the itineraries_to_legs list is used to define the constraint;
# for each leg ell, only add up those x[i]'s for which the itinerary i uses leg ell. 
leg_capacity_constrs = {}
for ell in range(nLegs):
    leg_capacity_constrs[ell] = m.addConstr( sum(x[i] for i in range(nItineraries) if ell in itineraries_to_legs[i]) <= B[ell])

# Specify the objective
m.setObjective( sum(revenue[i] * x[i] for i in range(nItineraries)), GRB.MAXIMIZE)

# Solve 
m.update()
m.optimize()

# Save the LP objective
LP_obj = m.objval

# Display the static allocation
print( [x[i].x for i in range(nItineraries)])
print( forecast)
print(LP_obj)

print( [leg_capacity_constrs[ell].slack for ell in range(nLegs)])
x_val = [x[i].x for i in range(nItineraries)]
seats_filled = [sum([x_val[i] for i in range(nItineraries) if ell in itineraries_to_legs[i]]) for ell in range(nLegs)]
print("Seats filled: ", seats_filled)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-25
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 4 rows, 8 columns and 12 nonzeros
Model fingerprint: 0x97cd214a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 4e+02]
  Bounds range     [6e+01, 1e+02]
  RHS range        [2e+02, 2e+02]
Presolve time: 0.00s
Presolved: 4 rows, 8 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4330000e+05   3.343750e+01   0.000000e+00      0s
       5    1.3240000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.324000000e+05
[60.0, 67.5, 60.0, 67.5, 97.5, 2.5, 2.5, 90.0]
[ 60.   67.5  60.   75.   97.5 105.   90.   90. ]
132400.0
[0.0, 0.0, 0.0, 0.0]
Seats filled: 

##  Solving the dynamic multi-leg capacity control problem

In the code below, we will use the LP formulated above as part of a dynamic bid-price control policy for determining which itinerary requests to accept or reject.

In [2]:
# First, we will define a function to repeatedly solve
# the LP at different periods and with different numbers of remaining seats. 
def bpc(b,t):
    for ell in range(nLegs):
        leg_capacity_constrs[ell].rhs = b[ell]
    
    for i in range(nItineraries):
        x[i].ub = (T - t)*probability[i]
    
    m.update()
    m.optimize()
    
    dual_val = [leg_capacity_constrs[ell].pi for ell in range(nLegs)]
    
    return dual_val

# To make the output easier to read, let's also disable the output from Gurobi:
m.Params.outputflag = 0

# Next, we will define a loop to carry out the simulation.

# First, we need to set up a proper probability mass function. The code
# below is the same as in the single leg simulation:
# Create a new array, probability_aug, which corresponds to a true probability
# mass function:
probability_aug = np.zeros(nItineraries+1)
probability_aug[0:nItineraries] = probability # First nItineraries elements are the same as probability
probability_aug[nItineraries] = 1 - sum(probability) # Last element is one minus the rest.

# Next, we have the loop for the simulation.
nSimulations = 100
np.random.seed(1)

results_revenue = np.zeros(nSimulations)
results_remaining_seats = np.zeros( (nSimulations, nLegs) )

for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy()
    arrival_sequence = np.random.choice(range(nItineraries+1), T, p=probability_aug)
    
    for t in range(T):
        # Stop if all seats have been sold:
        if ((b == 0).all()):
            break
        
        i = arrival_sequence[t]
        
        if (i < nItineraries):
            dual_val = bpc(b,t)
            
            # Compute the total bid price of the request:
            total_bid_price = sum([dual_val[ell] for ell in itineraries_to_legs[i]])
            
            # If the revenue is at least the total bid price, and there is at least one
            # seat on each leg of this itinerary ...
            if ( (revenue[i] >= total_bid_price) and (b[itineraries_to_legs[i]] > 0).all() ):
                # ... then accept the request!
                b[itineraries_to_legs[i]] -= 1
                total_revenue += revenue[i]
    
    results_revenue[s] = total_revenue
    results_remaining_seats[s,:] = b
    

mean_LP_revenue = results_revenue.mean()
mean_LP_remaining_seats = results_remaining_seats.mean(axis = 0)

print("Mean LP policy revenue: ", mean_LP_revenue)
print("Mean LP policy seats remaining: ", mean_LP_remaining_seats)
print("LP objective: ", LP_obj)
print("Optimality gap: ", 100*(1 - mean_LP_revenue / LP_obj) )
    

Mean LP policy revenue:  130298.2
Mean LP policy seats remaining:  [0.34 0.33 0.12 0.09]
LP objective:  132400.0
Optimality gap:  1.5874622356495505


## Simulating the myopic policy

Let's also see how the myopic policy does. 

In [3]:
np.random.seed(1)

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_seats = np.zeros( (nSimulations, nLegs) )

for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy()
    arrival_sequence = np.random.choice(range(nItineraries+1), T, p=probability_aug)
    
    for t in range(T):
        # Stop if all seats have been sold:
        if ((b == 0).all()):
            break
        
        i = arrival_sequence[t]
        
        if (i < nItineraries):
            # If there is a free seat on each leg for this itinerary...
            if ( (b[itineraries_to_legs[i]] > 0).all() ):
                # ... accept the request!
                b[itineraries_to_legs[i]] -= 1
                total_revenue += revenue[i]
    
    results_myopic_revenue[s] = total_revenue
    results_myopic_remaining_seats[s,:] = b
    

mean_myopic_revenue = results_myopic_revenue.mean()
mean_myopic_remaining_seats = results_myopic_remaining_seats.mean(axis = 0)

print("Mean myopic revenue: ", mean_myopic_revenue)
print("Mean myopic seats remaining: ", mean_myopic_remaining_seats)
print("LP policy outperforms myopic by: ", 100*(mean_LP_revenue / mean_myopic_revenue - 1) )

print("How suboptimal is myopic (in the worst case?): ", 100*(1 - mean_myopic_revenue/LP_obj))

Mean myopic revenue:  124422.7
Mean myopic seats remaining:  [0.   0.05 0.06 0.  ]
LP policy outperforms myopic by:  4.722209050277804
How suboptimal is myopic (in the worst case?):  6.025151057401812
